In [406]:
import numpy as np
import pandas as pd

In [407]:
filename = 'data/소상공인시장진흥공단_상가(상권)정보_20211231/소상공인시장진흥공단_상가(상권)정보_서울_202112.csv'
df_seoul = pd.read_csv(filename)

In [408]:
coffee_seoul = df_seoul[df_seoul.상권업종중분류코드 == 'Q12']
coffee_seoul = coffee_seoul[['상호명', '시도명', '시군구명']]
coffee_seoul.head()

,상호명,시도명,시군구명
23,커피빈,서울특별시,종로구
99,요거프레소,서울특별시,도봉구
191,우성커피숍,서울특별시,양천구
198,버블베어,서울특별시,강서구
233,알뤼르,서울특별시,강남구


In [409]:
# 상호명이 영어인 경우 모두 소문자로 변경 후 영어만 남기기
coffee_seoul['상호명'] = coffee_seoul['상호명'].apply(lambda x: x.lower())
coffee_seoul.head(3)

,상호명,시도명,시군구명
23,커피빈,서울특별시,종로구
99,요거프레소,서울특별시,도봉구
191,우성커피숍,서울특별시,양천구


In [410]:
# 확인
coffee_seoul['상호명'].str.isupper().sum()

0

In [411]:
# 커피빈, 이디야, 스타벅스, 빽다방, 한글 영어 포함.
coffee_name = '커피빈|이디야|스타벅스|빽다방|coffeebean|ediyacoffee|starbucks|paikdabang'
coffee_seoul['상호명'] = coffee_seoul.loc[coffee_seoul['상호명'].str.contains(coffee_name)]
coffee_seoul.dropna(inplace=True)

In [412]:
coffee_seoul.head()

,상호명,시도명,시군구명
23,커피빈,서울특별시,종로구
390,커피빈코리아낙성대역점,서울특별시,관악구
1131,이디야커피,서울특별시,영등포구
1411,커피빈,서울특별시,강남구
1426,커피빈,서울특별시,강남구


In [413]:
# 상품명으로 변경
def change(_df):
    for i in _df.index:
        if _df.상호명[i].find('커피빈') != -1:
            _df.상호명[i] = '커피빈'
        elif _df.상호명[i].find('coffeebean') != -1:
            _df.상호명[i] = '커피빈'
        elif _df.상호명[i].find('이디야') != -1:
            _df.상호명[i] = '이디야'
        elif _df.상호명[i].find('ediyacoffee') != -1:
            _df.상호명[i] = '이디야'
        elif _df.상호명[i].find('스타벅스') != -1:
            _df.상호명[i] = '스타벅스'
        elif _df.상호명[i].find('starbucks') != -1:
            _df.상호명[i] = '스타벅스'
        elif _df.상호명[i].find('빽다방') != -1:
            _df.상호명[i] = '빽다방'
        elif _df.상호명[i].find('paikdabang') != -1:
            _df.상호명[i] = '빽다방'
    return _df


In [414]:
# 모든 장소를 받을 함수
coffee_name = '커피빈|이디야|스타벅스|빽다방|coffeebeans|ediyacoffee|starbucks|paikdabang'
def coffee(df):
    coffee_df = df[df.상권업종중분류코드 == 'Q12']
    coffee_df = coffee_df[['상호명', '시도명', '시군구명']]
    coffee_df['상호명'] = coffee_df['상호명'].apply(lambda x: x.lower())
    coffee_dfs = coffee_df.loc[coffee_df['상호명'].str.contains(
        coffee_name)]
    coffee_dfs.dropna(inplace=True)
    coffee_dfs.reset_index(inplace=True)
    coffee_dfs.drop(columns=['index'], inplace=True)
    coffee_dfs = change(coffee_dfs)
    return coffee_dfs

In [415]:
# 모든 파일을 한번에 불러오자. (glob)
import warnings
warnings.filterwarnings('ignore')
from glob import glob

In [416]:
df_list = []
for file in glob('../07.Cartogram/data/소상공인시장진흥공단_상가(상권)정보_20211231/소상공인시장진흥공단*.csv'):
    gs = pd.read_csv(file)
    df_list.append(gs)
df = pd.concat(df_list)
coffee_df = coffee(df)
coffee_df.head(3)

,상호명,시도명,시군구명
0,스타벅스,강원도,강릉시
1,이디야,강원도,원주시
2,스타벅스,강원도,춘천시


In [417]:
coffee_df.shape

(4560, 3)

In [418]:
coffee_df.to_csv('data/각 지역별 커피 매장.csv', index=False)

In [419]:
coffee_df = pd.read_csv('data/각 지역별 커피 매장.csv')

# 각매장의 ID값이 필요... pivot table에 value가 없다...
coffee_df['Id'] = coffee_df.groupby('상호명').ngroup()
coffee_df.head()

,상호명,시도명,시군구명,Id
0,스타벅스,강원도,강릉시,1
1,이디야,강원도,원주시,2
2,스타벅스,강원도,춘천시,1
3,이디야,강원도,춘천시,2
4,스타벅스,강원도,춘천시,1


In [420]:
# 각 지역 매장수
pop = pd.pivot_table(coffee_df,
                    values = ['Id'],
                     index = ['시도명', '시군구명'],
                     columns = ['상호명'],
                     aggfunc = 'count'
)
pop.head()

Id                
상호명       빽다방 스타벅스   이디야  커피빈
시도명 시군구명                     
강원도 강릉시   2.0  4.0  17.0  NaN
    고성군   NaN  1.0   NaN  NaN
    동해시   1.0  1.0   1.0  1.0
    삼척시   1.0  NaN   5.0  NaN
    속초시   NaN  4.0   9.0  NaN

In [421]:
pop.columns.get_level_values(0)

Index(['Id', 'Id', 'Id', 'Id'], dtype='object')

In [422]:
pop.columns.get_level_values(1)

Index(['빽다방', '스타벅스', '이디야', '커피빈'], dtype='object', name='상호명')

In [423]:
# 컬럼명 변경: 다중 인덱스 -> 단일 인덱스
new_columns = []
for i in range(len(pop.columns.get_level_values(0))):
    new_columns.append(pop.columns.get_level_values(1)[i])
pop.columns = new_columns
pop.head(3)

빽다방  스타벅스   이디야  커피빈
시도명 시군구명                      
강원도 강릉시   2.0   4.0  17.0  NaN
    고성군   NaN   1.0   NaN  NaN
    동해시   1.0   1.0   1.0  1.0

In [424]:
pop.reset_index(inplace=True)
pop.head(3)

,시도명,시군구명,빽다방,스타벅스,이디야,커피빈
0,강원도,강릉시,2.0,4.0,17.0,NaN
1,강원도,고성군,NaN,1.0,NaN,NaN
2,강원도,동해시,1.0,1.0,1.0,1.0


In [425]:
# NaN의 값을 0으로 바꾸자.
pop.fillna(0, inplace=True)
pop.tail(3)

,시도명,시군구명,빽다방,스타벅스,이디야,커피빈
241,충청북도,청주시 청원구,5.0,3.0,8.0,0.0
242,충청북도,청주시 흥덕구,6.0,8.0,17.0,0.0
243,충청북도,충주시,2.0,4.0,8.0,0.0


In [426]:
# 시군구명 단어 2개 --> 1개
pop['시군구명'] = pop['시군구명'].apply(lambda x: x.split()[1] if len(x.split()) == 2 else x)

In [429]:
# 빠진 데이터 채우기
pop.loc[244] = ['경상북도', '의성군', 0, 0, 0, 0]
pop.loc[245] = ['경상북도', '군위군', 0, 0, 0, 0]
pop.loc[246] = ['경상남도', '의령군', 0, 0, 0, 0]

In [430]:
pop.to_csv('data/각 지역별 커피 매장수_전처리완료.csv', index=False)